# Generate bitcoin addresses for Irving & Holden's 2016 clinical trial word document

### By [Daniel Himmelstein](http://dhimmel.com/) ([@dhimmel](https://twitter.com/dhimmel))

This notebook computes the bitcoin addresses for the [word document](https://f1000researchdata.s3.amazonaws.com/datasets/8114/9c9f9a18-a852-40c6-953e-c75107abc714_Appendix_1_-_unformatted_text_file_.docx) from the following study:

> **How blockchain-timestamped protocols could improve the trustworthiness of medical science** [[version 2; referees: 3 approved]](https://doi.org/b2pt)<br>
Greg Irving, John Holden<br>
_F1000Research_ (2016) DOI: [10.12688/f1000research.8114.2](https://doi.org/10.12688/f1000research.8114.2)

It uses the method described by Benjamin Gregory Carlisle in a 2014 blog post titled [Proof of prespecified endpoints in medical research with the bitcoin blockchain](http://www.bgcarlisle.com/blog/2014/08/25/proof-of-prespecified-endpoints-in-medical-research-with-the-bitcoin-blockchain/).

**Warning: the Carlisle method is not the recommended approach for _proof of existence_ using Bitcoin. This notebook is not an endorsement of the method, but rather a demostration that the address generation in the Irving & Holden study is flawed.**

## Dependencies

This is a Python 3 notebook. It requires [Python Bitcoin Tools](https://github.com/vbuterin/pybitcointools), which can be installed with `pip install bitcoin`. This notebook was generated using `bitcoin==1.1.42` from [PyPI](https://pypi.python.org/pypi/bitcoin/1.1.42).

In [1]:
from urllib.request import urlopen
import hashlib

import bitcoin

## Generate the private key

Get the sha256 hash for [Dataset 1. Unformatted text file.
](https://doi.org/10.5256/f1000research.8114.d114596 "Irving & Holden. F1000 Research. Dataset 1. Unformatted text file.")

In [2]:
url = 'https://f1000researchdata.s3.amazonaws.com/datasets/8114/9c9f9a18-a852-40c6-953e-c75107abc714_Appendix_1_-_unformatted_text_file_.docx'
response = urlopen(url)
data = response.read()
checksum = hashlib.sha256(data)
private_key = checksum.hexdigest()
private_key

'8da3088936035521f9e9b57963679d89e306a06c6aebd1167b4d198e79562326'

In [3]:
# Get the private key's format
bitcoin.get_privkey_format(private_key)

'hex'

## Generate the corresponding public keys

There are two common types of bitcoin public keys (compressed and uncompressed) that result in different bitcoin addresses. Neither Carlisle or Irving & Holden report which type of public they use, so we'll try both.

In [4]:
# Uncompressed public key
public_key = bitcoin.privkey_to_pubkey(private_key)
public_key

'04a1582612a51aa8cea8e8ced2078d01141ff941e6c5d985bbae2536ce33ef5396bc7946f188aeb99d6b575b935c218ae19780ef77a535107a5272e7390e1001e4'

In [5]:
# Compressed public key
public_key_compressed = bitcoin.compress(public_key)
public_key_compressed

'02a1582612a51aa8cea8e8ced2078d01141ff941e6c5d985bbae2536ce33ef5396'

## Generate the corresponding addresses

Note that neither address matches the address reported by Irving & Holden, which was [`1AHjCz2oEUTH8js4S8vViC8NKph4zCACXH`](https://blockchain.info/address/1AHjCz2oEUTH8js4S8vViC8NKph4zCACXH).

In [6]:
# Uncompressed address
address = bitcoin.pubkey_to_address(public_key)
address

'1P6cxmuSsjDqUGCsyaEzgcj7iTEPsMAjhU'

In [7]:
# Compressed address
address_compressed = bitcoin.pubkey_to_address(public_key_compressed)
address_compressed

'17pJjJGJJTzVsJx9JSfbx6vp1sGkPNoDoA'

In [8]:
# Check whether the Irving & Holden address is wrong
address_irving = '1AHjCz2oEUTH8js4S8vViC8NKph4zCACXH'
if not address_irving in {address, address_compressed}:
    print('Irving & Holden have a big problem.')

Irving & Holden have a big problem.


## Check whether either of the correct addresses has ever been used

As of March 6, 2017, neither address has been used.

In [9]:
# URLs for blockchain.info address details
for address in address, address_compressed:
    url = 'https://blockchain.info/address/{}'.format(address)
    print(url)

https://blockchain.info/address/1P6cxmuSsjDqUGCsyaEzgcj7iTEPsMAjhU
https://blockchain.info/address/17pJjJGJJTzVsJx9JSfbx6vp1sGkPNoDoA


## Alternative implementation

For an altertative implementation, you can generate the sha256 checksum via the unix shell:

```sh
URL=https://f1000researchdata.s3.amazonaws.com/datasets/8114/9c9f9a18-a852-40c6-953e-c75107abc714_Appendix_1_-_unformatted_text_file_.docx
curl --silent $URL | shasum --algorithm 256
```

Then you can use [bitaddress.org](https://www.bitaddress.org) to generate the bitcoin addresses. Just go to the "Wallet Details" page and paste the sha256 hash into the "Enter Private Key" field. This approach generates the same addresses as this notebook.